## Unpaired Translation with CycleGAN

### Unpaired Image-to-Image Translation

이 비디오에서는 짝을 이루지않은(unpaired) 이미지를 이미지로 변환하는 방법을 배웁니다. 먼저 Pix2Pix 에서 본 짝을 이루는 이미지 변환과 짝을 이루지 않은 변환에서의 변형을 비교하겠습니다. 그런 다음 두 이미지 스타일 더미 간의 매핑이 작동하는, 쌍을 이루지 않은 이미지간의 변환 방식을 볼 수 있습니다. 실제로 이것은 두 가지 파일 더미의 공통점과 차이점을 찾는 것입니다.
<br>
<img src="./images/C3W3.01.png" width="700">
<br>

이전에 짝을 이루는 이미지에서 이미지로 변환하는 것을 보았고, 그 때 가장자리 윤곽과 현실적인 핸드백 사이에 명확한 입력, 출력 쌍이 있었습니다. 가장자리 감지기를 사용하여 사실적인 이미지에서 해당 가장자리 윤곽으로 갈 수 있기 때문에 짝을 이룬 데이터 세트를 매우 쉽게 얻을 수 있습니다. 즉, 짝을 이루는 훈련 데이터를 얻기 위한 알고리즘이 있거나, 어떤 이유로 이미 짝을 이룬 훈련 데이터가 있는 경우에는 좋습니다. 하지만, **짝을 이루지 않은 이미지를 이미지로 변환하는 것은 그렇게 간단하지 않습니다.** 말을 얼룩말로 바꾸고 싶거나 여기의 사진으로부터 모네의 그림을 갖고 싶다고 가정해 봅시다. 이 생성은 훨씬 더 어렵습니다. 왜냐하면 모든 사진에 대해 모네가 그린 것처럼 보이는 그림이 없으며, 그 반대의 경우도 마찬가지입니다. 그곳에는 수천 개의 훈련 쌍이 없을 것입니다.
<img src="./images/C3W3.02.png" width="700">
<br>

이러한 **짝을 이루는 이미지에서 이미지로의 변환 작업** 사이의 패러다임은 이미지 각각에 대해 이러한 **쌍을 이루는 이미지**가 있다는 것입니다. $x_i$와 $y_i$가 짝을 이루게되고, 여기서 $i$는 아마도 0, 1, 2일 것입니다. 서로 일치하는 이 모든 다른 쌍들이 있지만, 항상 이 대응을 가질 필요는 없읍니다. 
**짝을 이루지 않은 이미지에서 이미지로의 변환**에서는, 실제로 $X$와 $Y$라는 **두 가지 다른 이미지 스타일의 두 더미**가 있습니다. 하나는 사실적인 사진이고 다른 하나는 모네나 세잔 또는 다른 사람의 작품일 수 있습니다. 또는 일대일 대응이 전혀 없는 겨울 장면 대 여름 장면의 더미, 또는 말 더미 대 다른 얼룩말 더미를 가질 수 있습니다. $X$와 $Y$에서 쌓인 이 두 더미를 사용하여 모델이 한 더미에서 다른 더미로 일반적인 스타일 요소를 배우고 이미지를 서로 변환하고 때로는 그 반대로 변환하기를 원합니다.
<img src="./images/C3W3.03.png" width="700">
<br>

여기에서 사진으로부터 모네를 얻을 수도 있고,사진으로부터 반 고흐를 얻을 수도 있고, 반대로 세잔과 우키요에의 작품으로부터 사진을 얻을 수도 있습니다. 하지만 여기서 중요한 것은 양귀비 밭의 사진을 마치 모네가 그린 것처럼 보이게 만들고 있다는 것입니다. 하지만 이 모네에서는 여전히 양귀비 밭입니다. 아직 **일부 유형의 콘텐츠가 보존**되어 있습니다. **변경된 것은 스타일적인 요소**일 뿐입니다. 이러한 변환 작업을 생각하는 데 있어서 각 더미에 대한 고유한 공통점과 스타일적인 것을 인식하는 것이 매우 중요합니다. 즉, 공통점을 찾아내고 공통 요소를 유지하고, 각각의 요소에 해당하는 고유한 요소만 다른 더미로 이전하는 것이기 때문입니다.
<img src="./images/C3W3.04.png" width="700">

구체적으로 말하면 여기 얼룩말이나 말이 있으면, 얼룩말 더미가 있고 이 얼룩말 이미지에서 생성하려는 말은 여전히 같은 방향에 있어야 합니다. 당신은 단지 그 줄무늬가 사라지기를 원합니다. 모델의 목표는 이 두 말과 얼룩말 더미를 매핑하여 **공통 요소와 고유 요소를 파악**하는 것입니다. **공통 요소는 종종 콘텐츠**로 알려져 있습니다. 이 두 더미에 공통적인 것이 이미지의 콘텐츠이고, 스타일은 종종 그 더미들 사이의 차이점을 나타냅니다. 다시 말하지만, 여기의 **내용(contents)은 말이나 얼룩말의 일반적인 모양과 배경이** 될 겁니다. 아마도 **스타일은 여기 말의 단색과 단순한 패턴과  얼룩말의 분명한 줄무늬**일 것입니다. 이번 주 다음 비디오에서 이것이 어떻게 수행되는지 배우게 될 것입니다.
<br>
<img src="./images/C3W3.05.png" width="700">

요약하면, 짝을 이루지 않은 이미지에서 이미지로의 변환은 짝을 이룬 이미지 대신 서로 다른 스타일의 이미지 더미를 사용합니다. 모델은 두 더미에 모두 있는 콘텐츠를 유지하면서도, 각각의 더미가 가진 서로 다른 고유한 스타일로 변경하는 두 더미 간의 매핑을 학습합니다.
<br>
<img src="./images/C3W3.06.png" width="700">

### CycleGAN Overview
이 비디오에서는 짝을 이루지 않은 이미지 대 이미지 변환 문제를 해결하는 CycleGAN에 대해 배웁니다. CycleGAN의 'Cycle'이 무엇을 의미하는지, 그리고 이것이 짝을 이루지 않은 이미지 대 이미지 변환에서 어떻게 작용하는지 알게 될 것입니다. 또한 CycleGAN을 구성하는 두 개의 GAN을 볼 수 있습니다. 이 GAN은 정말 멋지고 일반적으로 GAN에 대해 어떻게 생각해야 하는지 보여주며, GAN은 다양한 방법으로 구성할 수 있음을 알려줍니다.
<img src="./images/C3W3.07.png" width="700">
<br>

이 시점에서 페어링되지 않은 이미지 변환은 불가능하게 들릴 수 있습니다. 이미지 쌍이 없는데 어떻게 생성해야 하는지 모델이 어떻게 알 수 있을까요? 얼룩말을 어떻게 말처럼 보이는 것으로 바꿔야 할까요?
<img src="./images/C3W3.08.png" width="700">
<br>

CycleGAN이 작동하도록 만드는 정말 멋진 통찰력이 있습니다. 그것은 실제 얼룩말들의 한 더미에서 한 이미지를 가지고, 다른 더미의 말 스타일로 변환한 다음, 다시 얼룩말로 변환하지만 여전히 가짜인 경우입니다. 또 다른 가짜 얼룩말을 생성하기 위해 이 가짜 말을 받아들이고 있습니다. 글쎄요, 멋진 통찰력은 기술적으로 이미지의 내용이 아니라 스타일만 변경하기 때문에 이 두 이미지가 동일해야 한다는 것입니다. 이것은 픽셀 거리 손실에 대해 다시 생각하게 할 수 있습니다.
<img src="./images/C3W3.09.png" width="700">
<br>

한 더미에서 다른 더미로 매핑하고 다시 매핑하는 동안 이러한 콘텐츠 보존은 변환이 이 두 더미 사이에 싸이클을 형성하기 때문에 싸이클 일관성(consistency)으로 알려져 있습니다. 
<img src="./images/C3W3.10.png" width="700">
<br>

이 싸이클을 생성하는 한 가지 간단한 방법은 실제로 두 개의 서로 다른 GAN을 사용하는 것입니다. 하나는 얼룩말에서 말로 가고, 다른 하나는 반대 방향으로 말에서 얼룩말로 가는 것입니다. 이 두 GAN은 함께 싸이클 일관성을 가지고 있어 현실감 있는 짝을 이루지 않은 변환을 가능하게 합니다. GAN의 **적대적인 부분은 둘 다에 대해 구별하고 이미지의 사실감을 보장**하는 반면, **싸이클 일관성 부분은 실제로 스타일을 이동하면서 콘텐츠를 보존**하는 역할을 합니다.
<img src="./images/C3W3.11.png" width="700">
<br>

앞의 예에서 GAN Z-H, 또는 얼룩말에서 말로를 사용하여 먼저 가짜 말을 생성합니다. 그런 다음 두 번째 GAN H-Z, 또는 말에서 얼룩말로를 사용하여 가짜 말에서 가짜 얼룩말을 생성합니다. 그러면 반대도 가능할 것입니다. 그 말을 먼저 얼룩말로 갔다가 다시 반대 방향으로 갈 것입니다. 둘 다 해당 GAN을 모두 사용하고 해당 생성기를 모두 사용합니다.
<img src="./images/C3W3.12.png" width="700">
<br>

이러한 GAN의 아키텍처는 이미 보고 배운 것입니다. 따라서 Pix2Pix 에서와 같이 판별자는 PatchGAN입니다. 그러나 여기에는 물론 두 개의 GAN이 있으므로 두 개의 판별자가 있으므로 단일 값 대신 행렬의 값으로 주어지는 이미지와 출력의 패치를 보는 두 개의 PatchGAN이 있습니다.
<img src="./images/C3W3.13.png" width="700">
<br>

그리고, 생성기는 스타일 전송의 이미지 초해상도에 대한 이전 작업에서 가져옵니다. 따라서 기본적으로 pix2pix 과 DCGAN의 두 유닛이 모두 사용됩니다. CycleGAN은 기본적으로 U-Net에서 개념을 차용합니다. 다운샘플링과 인코딩 섹션이 있고 다음 디코딩 섹션과 업샘플링이 있습니다. 이러한 다양한 인코딩 및 디코딩 상자는 BatchNorm과 ReLu가 있는 컨볼루션 레이어로 구성되어 있습니다.
<img src="./images/C3W3.14.png" width="700">
<br>

이 U-Net 프레임워크 외에도 CycleGAN은 DCGAN 생성기에서와 같이 더 많은 컨볼루션 레이어로 병목 섹션을 확장합니다. CycleGAN의 생성기는 이미지를 입력으로, 이 $x$가 입력,  사용하기 때문에 일반적인 유닛이 여기에서도 실제로 잘 수행했을 가능성이 큽니다. 확실히 이 새로운 아키텍처로 구성하지만, 이것이 이 전체를 작동하게 하는 측면에서 제한 요소가 아닐 수도 있습니다. 그것은 단지 저자들이 찾은 꽤 유용한 것으로 밝혀진 개선일 뿐입니다. 
<img src="./images/C3W3.15.png" width="700">
<br>

여기의 병목 섹션에서도 ResBox 또는 잔여 연결이라고 하는 자체 내에서 추가 건너뛰기 연결을 사용합니다. 이것들은 단순한 건너 뛰기 연결입니다. 나중에 이것들에 대해 더 배우게 될 것입니다. 모델이 항등 함수를 학습할 수 있도록 추가 레이어 및 이미지 변환을 추가하는 데 도움이 됩니다.
<img src="./images/C3W3.16.png" width="700">
<br>

요약하면, CycleGAN은 두 더미의 이미지를 서로 변환하는 두 개의 서로 다른 GAN으로 구성됩니다. 이것은 짝을 이루지 않은 이미지 대 이미지 변환을 위한 것입니다. 판별자는 PatchGAN에서 가져온 것이고 생성기는 DCGAN과 U-Net에서 얻은 학습과 추가 건너뛰기 연결의 조합입니다.
<img src="./images/C3W3.17.png" width="700">

### CycleGAN : Two GANs

이 비디오에서는 이 두 GAN과 CycleGAN이 서로 어떻게 상호 작용하는지 살펴보겠습니다. 2개의 GAN에서 예상한 대로 4개의 구성요소, 2개의 생성기와 2개의 판별기가 있습니다.
<img src="./images/C3W3.18.png" width="700">
<br>

먼저 얼룩말에서 말로 이동하는 GAN이 있고, 말에서 얼룩말로 이동하는 GAN이 있습니다. 그러면 네 가지 구성 요소는 얼룩말에서 말에 대한 매핑을 학습하는 생성기이고, 다른 쪽 끝에는 말의 이미지가 얼마나 사실적인지 확인하는 판별기가 있습니다. 그런 다음 말에서 얼룩말로 반대 방향으로의 한쌍이 더 있어, 매핑을 하는 생성기와 해당 얼룩말이 얼마나 사실적인지 확인하는 판별자가 있습니다.
<img src="./images/C3W3.19.png" width="700">
<br>

한 방향에서 시작하면 얼룩말 이미지를 가져와 얼룩말에서 말 생성기를 사용하여 말에 매핑합니다. 그런 다음 말 판별자는 그 말 더미에서 말의 실제 이미지와 들어오는 가짜 말을 보고 그들이 얼마나 진짜인지 가짜인지 판단하는 역할을 합니다. 물론 이것은 패치 GAN이므로 이미지에서 보이는 패치의 분류 행렬을 출력하고 있습니다.
<img src="./images/C3W3.20.png" width="700">
<br>

반대 방향도 비슷합니다. 말에서 얼룩말로 가는 것뿐이므로 판별자는 얼룩말과 얼룩말이 얼마나 현실적인지에 초점을 맞출 것입니다.
<img src="./images/C3W3.21.png" width="700">
<br>

요약하면, CycleGAN은 4개의 구성요소로 구성되어 있으며, 각각 Generator와 Discriminator가 있는 2개의 GAN이며, Generator와 Discriminator의 입력은 짝을 이루는 실제 이미지가 없다는 점을 제외하고는 Pix2pix와 동일합니다. 따라서 대상 출력이 없기 때문에 실제 대상 출력에서 이전에 보았던 추가 픽셀 거리 손실이 없으며 대신 두 개의 서로 다른 이미지 더미를 보고 있습니다.
<img src="./images/C3W3.22.png" width="700">


### CycleGAN : Cycle Consistency

이 비디오에서는 CycleGAN에 손실 항을 넣어 순환(Cycle) 성질을 주는 순환 일관성(Cycle Consistency)에 대해 배웁니다. **순환 일관성**은 손실 함수에 대한 **추가 손실 항**이며, 이는 두 GAN 각각에 대한 것이며 싸이클 일관성이 정확히 어떻게 도움이 되며, CycleGAN에 포함하지 않을 때 어떤 일이 발생하는지 살펴보겠습니다.
<img src="./images/C3W3.23.png" width="700">
<br>

먼저 얼룩말에서 말로, 그리고 다시 얼룩말로의 방향을 살펴보겠습니다. 얼룩말로부터 말 생성기는 얼룩말의 실제 이미지를 가짜 말에 매핑하고, 다른 생성기는 해당 말을 다시 얼룩말에 매핑합니다. 싸이클 일관성이 기대하는 것은 생성된 가짜 얼룩말이 실제 얼룩말과 똑같이 보이도록 하는 것입니다. 스타일만 변경되었었어야 하기 때문입니다. 여기서 할 수 있는 것은 이 두 이미지 간의 픽셀 차이를 가져와 손실 함수에 추가할 수 있고 이 두 이미지가 가능한 한 가깝게 되도록 권장하는 것입니다.
<img src="./images/C3W3.24.png" width="700">
<br>

이것은 반대 방향으로도 적용됩니다. 말을 얼룩말에 매핑하고 다시 말에 매핑하고 이 둘 사이의 픽셀 차이를 다시 가져오려고 합니다.
<img src="./images/C3W3.25.png" width="700">
<br>

이제 양방향의 픽셀 차이를 합산하여 전체 순환 일관성 손실을 구성할 수 있습니다. 하나는 얼룩말에서 말에서 얼룩말로 가는 것입니다. 순환을 거친 후 진짜 얼룩말과 가짜 얼룩말의 차이를 보는 것입니다. 말에서 얼룩말로 그리고 다시 말 방향으로 가는 것도 마찬가지입니다. 각각의 i개 샘플을 합산하면 됩니다. 이것은 생성기들의 **전체 순환 일관성 손실 항**을 구성합니다.
<img src="./images/C3W3.26.png" width="700">
<br>

이것을 적대적 손실에 추가할 때, 여기에서 볼 수 있듣이 두 생성기를 사용하기 때문에 실제로는 두 생성기 모두에 대해 하나의 옵티마이저만 있습니다. 두 생성기에 모두 사용되는 손실 항은 단 하나뿐입니다. 이는 이 순환 일관성 손실로부터 계산되고 두 생성기 간에 공유됩니다. 다시 말하지만, 적대적 손실은 주된 GAN 손실 함수가 아닙니다. 이전에 본 것과 약간 다르기 때문에 CycleGAN의 적대적 손실 함수가 정확히 무엇인지 배우게 될 것입니다.
<img src="./images/C3W3.27.png" width="700">
<br>

보다 구체적으로 말하면 훈련 데이터 세트의 얼룩말과 말에 대한 순환 일관성 손실을 합산하고, 이를 어떤 Lambda 항으로 가중치를 부여하여 전체 싸이클 일관성 손실 항을 얻은 다음, 생성기 손실에 추가할 수 있습니다. 한 가지 재미있는 사실은 순환 일관성이 CycleGAN에서 사용되는 것보다 더 광범위한 개념이라는 것입니다. 실제로 딥 러닝 전반에 걸쳐 꽤 많이 사용됩니다. 예를 들어 데이터 증강에 도움이 되었으며, 프랑스어에서 영어로, 그리고 다시 프랑스어로 텍스트 번역에도 사용되었습니다. 동일한 문구를 다시 얻을 것으로 예상하지만, 그렇지 않고 동일한 문구를 다시 얻지 못하면 이를 데이터 세트를 보강하는 데 사용되며, 이에는 데이터 세트에 약간의 노이즈가 추가됩니다. 순환 일관성으로 할 수 있는 것들 중에 꽤 멋진게 있습니다..
<img src="./images/C3W3.28.png" width="700">
<br>

CycleGAN 논문은 또한 멋진 제거(ablation) 연구와 제거 연구가 무엇인지 보여주었습니다. 제거라는 단어는 CycleGAN에서 소개하는 다양한 구성 요소를 잘라내고 이러한 다양한 구성 요소 없이 모델이 어떻게 작동하는지 확인하는 것을 의미합니다. 첫째, CycleGAN을 사용했지만 GAN 구성 요소를 제거했습니다. 순환 일관성 손실만 있다면 이 모델은 어떻게 될까요? 순환 일관성 손실이 그다지 좋지 않다는 것을 알 수 있습니다. 이 이미지는 그다지 좋지 않습니다. 여기서 일어나는 일은 실제로 쌍을 이룬 데이터 세트를 보고 있다는 것입니다. CycleGAN은 짝을 이루지 않은 상태에서 작동하므로 실제로는 두 개의 더미만 보고 있다는 것을 기억하십시오. 페어링을 전혀 고려하지 않습니다. 그러나 여기서 쌍을 이룬 데이터 세트를 사용하는 이유는 출력이 실제와 얼마나 멀리 떨어져 있는지를 파악하기 위한 것입니다. 짝이 없는 변환을 보고 있더라도 모델의 제거 연구를 평가하고 살펴보는 아주 멋진 방법은 짝을 이루는 변환 데이터 세트를 사용하는 것입니다. 여기서 진행되는 것은 이 사실적인 이미지를 가져와서 이 분할맵 출력을 생성하거나 반대 방향으로, 분할맵 출력을 가져와서 이 사실적인 이미지를 생성한다는 것입니다. 이들은 여기에서 다른 방향으로 동일한 쌍이며, 두 번째 세트도 다른 방향으로 동일한 쌍입니다. 순환 일관성만으로는 이러한 출력이 전혀 현실적이지 않다는 것을 알 수 있습니다. 완전히 검은색 사각형이 표시되고 아마도 일부 모드 붕괴가 진행 중일 것이며 현실감이 별로 없습니다. GAN의 적대적 손실은 실제로 상황을 사실적으로 보이게 하므로 예상대로 GAN 없이 순환 일관성만 있으면 원하는 현실적인 출력을 얻을 수 없습니다.
<img src="./images/C3W3.29.png" width="700">
<br>

그러나 GAN만 있고 순환 일관성이 없는 경우에는 어떻게 됩니까? 글쎄, 출력은 실제로 상당히 사실적으로 보입니다. 약간의 모드 붕괴가 일어나고 있다는 점을 제외하고는 꽤 괜찮아 보입니다. 이 둘은 매우 다른 입력 이미지를 가지고 있지만 이들로부터 생성된 분할 마스크는 매우 유사하고 약간의 모드 붕괴가 진행 중이며 이는 분명히 이상적이지 않습니다. 이 사실적인 이미지는 검은색 사각형보다 훨씬 더 사실적이지만, 여전히 약간의 모드가 붕괴되어 있는 것처럼 보이며, 모든 동일한 특징들을 가지고 있고, 실제 이미지만큼 다양하지는 않습니다. 
<img src="./images/C3W3.30.png" width="700">
<br>

흥미롭게도 순환 일관성의 절반을 사용할 수 있지만, GAN이 다양하고 양질의 매핑을 학습하는 데에는 충분하지 않습니다. 순환 일관성 손실이 없는 경우 출력이 여전히 약간의 모드 붕괴가 표시되는 것을 볼 수 있습니다.
<img src="./images/C3W3.31.png" width="700">
<br>

그러나 CycleGAN이 말하려고 하는 것은 두 손실 조건, 즉 적대적 손실 조건과 양방향으로의 순환 일관성 손실이 모두 있다는 것입니다. 그런다면 이러한 출력에 정말 도움이 됩니다. 여기에서 진행 중인 모드 붕괴가 적고 출력의 다양성이 표시되며 여기에서 넘어가는 것처럼 상당히 사실적으로 보입니다.  CycleGAN은 두 세계의 장점을 실제로 사용하며 이러한 제거 연구를 통해 이 중 하나를 제거해도 효과가 없음을 보여줍니다. 이것이 바로 CycleGAN이 이것이 우리에게 정말로 필요한 것이며 중요한 기여라고 말하려고 하는 방법입니다. 연구 논문에서 제거 연구를 꽤 자주 볼 수 있습니다.
<img src="./images/C3W3.32.png" width="700">
<br>

요약하자면, 순환 일관성은 해당 이미지에서 공통 콘텐츠를 유지하면서 공통 스타일 요소를 전송하는 데 중요하며, 이는 정말, 정말 중요한 손실 항입니다. 이는 픽셀 거리 손실에 또는 적대적 손실에 추가하여 양방향에서 순환 일관성을 장려함으로써 수행할 수 있습니다. 제거 연구에 따르면 양방향의 순환 일관성 손실 항은 모드 붕괴를 방지하고, 이 거대하고 쌍을 이루지 않은 이미지들에서 이미지들로의 변환 작업을 돕는 데 도움이 됩니다.
<img src="./images/C3W3.33.png" width="700">

### CycleGAN: Least Squares Loss

이 비디오에서는 최소 제곱 손실(Least Squares Loss)에 대해 알아봅니다. 먼저 최소 제곱 및 통계의 전반적인 개념을 살펴본 다음 GAN에서 최소 제곱 손실이 의미하는 바를 살펴봅니다. 이것은 적대적 손실이 될 것이므로 Cycle GAN에서 BCE 손실 또는 W 손실을 대체할 것입니다.
<img src="./images/C3W3.34.png" width="700">
<br>

최소 제곱 손실 및 GAN에 대한 빠른 배경 지식을 살펴봅니다. 이것은 WGAN-GP와 비슷한 시점에 나왔습니다. 그리고 이때는 GAN에서의 훈련 안정성이 큰 문제였던 시기였습니다. **최소 제곱 손실은 훈련 안정성을 돕기위해 사용**됩니다. 즉, BCE 손실에서 보았던 그라디언트 손실 문제와 이에 따른 모드 붕괴 및 기타 문제를 일으켜 최악의 결과로 학습을 끝내는 것을 사라지게 하는 경향이 있습니다. 자세히 알아보기 전에 특히 훈련 안정성에 도움이 되는 수많은 GAN 손실 함수가 있으며, 일부 데이터 세트는 실제로 동일한 모델에서도 한 손실 함수가 다른 손실 함수보다 더 나은 결과를 보이기 때문에 종종 사람들이 이를 경험적으로 실험합니다. 사람들이 "더 나은" 것에 대해 서로 다른 견해를 가지고 있기 때문에 이것이 편견을 유발할 수도 있음을 기억하십시오. 생각해야 할 것은 훈련 시간도 사람들이 GAN을 생각하는데 있어서 큰 구성 요소라는 것입니다. WGAN-GP가 계산이 조금 더 복잡하기 때문에 약간 조금 느린 것을 보셨을 수도 있습니다.  과제에서 이를 관찰했을 수도 있습니다. 또한 이러한 이유로 때때로 사람들이 다른 손실 함수보다 특정 손실 함수를 사용하도록 압박하기도 합니다.
<img src="./images/C3W3.35.png" width="700">
<br>

최소 제곱은 통계에서 나온 아주 간단한 개념입니다. 잔차 제곱의 합을 최소화하는 방법입니다. 이것이 의미하는 바는 해당 선과 모든 점 사이의 거리 제곱합이 가장 작은 가장 적합한 선을 찾으려고 한다는 것입니다. 다음은 세 개의 서로 다른 점이 있는 예이며 이를 가장 잘 표현하는 선을 맞추려고 합니다. 선에서 각 점의 거리를 찾아서 이를 수행할 수 있습니다. 먼저 선을 그리고 한 점에서 그 선까지의 거리를 찾습니다. 특히, 그들의 제곱 거리를 살펴보고 그 제곱 거리를 최소화하는 가장 적합한 선을 찾고 싶습니다. 이렇게 선을 그리면 여기 보라색으로 보이는 다른 선보다 훨씬 더 큰 제곱 거리를 갖게 됩니다. 시도한 각 선에 대한 총 거리 또는 비용으로 제곱합을 얻고 가장 적합한 선을 찾기 위해 제곱합을 최소화합니다. 이것이 바로 선과 각 점 사이에 보이는 거리의 제곱인 잔차 제곱의 합을 최소화함을 의미하는 것입니다. 모든 제곱의 합계를 구하고 해당 값을 최소화하면 가능한 가장 적합한 선을 얻을 수 있습니다. 이것은 몇 점에 걸쳐 가장 잘 맞는 선을 찾는 매우 간단한 개념입니다.
<img src="./images/C3W3.36.png" width="700">
<br>

이것이 GAN 영역으로 번역되는 방식은 라인이 이제 레이블이 되었다는 것입니다. 따라서 본질적으로 실제 또는 가짜 레이블입니다. 판별기의 관점에서 레이블 1은 진짜이고, 해당 선에 상대적인 점은 실제 이미지에 대한 판별기 예측입니다. 점는 예측이고 그 거리를 찾으려는 선은 본질적으로 하나의 레이블입니다. 예측에서 해당 레이블까지의 제곱 거리를 얻으려고 합니다.
<img src="./images/C3W3.37.png" width="700">
<br>

많은 다른 지점에서 이 작업을 수행하기를 원하므로 많은 실제 이미지 $X$에서 기대 값을 취할 수 있으며, 가짜 이미지의 경우 동일하지만 레이블이 0입니다. 이것은 그것들을 넣는 매우 일반적인 방법입니다. 물론 CycleGAN을 사용하면 G가 실제 이미지를 받아들이고 판별기가 가짜 이미지를 평가할 것으로 예상합니다. 그런 다음 동일한 작업을 수행했습니다. 여러 개의 가짜 예제에서 이 작업을 수행합니다. 이는 G에 전달된 여러 $z$ 값일 뿐입니다. 여기서 $z$는 아마도 다른 얼룩말 이미지일 것입니다. 
<img src="./images/C3W3.38.png" width="700">
<br>

<img src="./images/C3W3.39.png" width="700">
<br>

이 식에서 $-0$ 이기 때문에 $0$을 제거하여 이것을 단순화할 수 있습니다. 이것이 최소제곱 적대적 손실 함수를 사용하는 판별자 손실 함수입니다. 
<img src="./images/C3W3.40.png" width="700">
<br>

생성기 측면에서는 이전에 본 다른 경우와 매우 유사합니다. 생성기는 가짜 출력이 가능한 한 진짜처럼 보이길 원하므로 대신 출력이 얼마나 멀리 떨어져 있는지 확인하기를 원할 것입니다.
<img src="./images/C3W3.41.png" width="700">
<br>

요약하면, 이들은 최소 제곱 적대적 손실 아래의 판별자 및 생성자 손실 항입니다. 이것은 이전에 본 손실 함수, 즉 BCE 손실과 유사하게 보일 수 있습니다. 그러나 중요한 것은 이 경우 **손실이 BCE의 Sigmoid에서 보았던 것처럼 매우 평평하지 않다는 것**을 알게 될 것입니다. 이는 판별자 예측이 정확히 1이고, 가짜에 대하여는 정확히 0인 경우 평평하게 되기 때문에 모든 그라디언트 소실 문제를 일으키는 원인이 됩니다. 이 최소 제곱 손실을 사용하여 거리 제곱을 생각하면 그렇지 않게되어 이 그라디언트 소실 문제가 크게 발생하지 않습니다. 이 최소 제곱 손실 함수는 대체로 BCE 손실에서 보았던 소실 그라디언트 문제를 해결하는데 도움이되며, 이 문제가 해결되면 GAN에서 더 나은 훈련 안정성이 제공됩니다. 이 방법에 익숙할 수 있는 사람들에게 이것은 평균 제곱 오차 또는 MSE라고도 합니다. 그러나 MSE가 무엇인지 모른다고 걱정할 필요는 없습니다. 이미 알고 있는 주제에 이것을 연결하려고 하면 됩니다.
<img src="./images/C3W3.42.png" width="700">
<br> 

Cycle GAN의 맥락에서 이것은 $\lambda$와 함께 순환 일관성 손실 항이 있지만 적대적 손실은 방금 본 최소 제곱 손실입니다.

<img src="./images/C3W3.43.png" width="700">
<br>

요약하면 최소 제곱 손실과 제곱 거리를 계산하는 방법에 대해 배웠습니다. 이 손실은 Cycle GAN에서 적대적 손실로 사용되며, BCE 손실보다 포화 및 소실 그라디언트 문제가 적습니다. 포인트가 라인에 있을 때만 평평하기 때문에 예측이 완벽하다는 것을 의미합니다. 예를 들어 진짜는 1, 가짜는 0입니다. 다른 곳에서는 그 제곱 거리가 됩니다. 따라서 포화 문제가 없습니다. 또한 이 비디오에서 생성기와 판별기 모두에서 최소 제곱 손실에 대한 방정식을 보았습니다. 그러나 이것이 최종 손실 방정식이 아님을 기억하십시오. 이전에 배운 순환 일관성 손실 항도 있습니다. 다음 비디오에서도 추가 손실 항에 대해 실제로 배우게 됩니다.
<img src="./images/C3W3.44.png" width="700">
<br>

### CycleGAN : Identity Loss

이 비디오에서는 CycleGAN의 항등 손실(identity loss)에 대해 배웁니다. 먼저 항등 손실이 작동하는 방식에 대해 배우고 이것이 출력에 어떤 영향을 미치는지 확인할 것입니다. 항등 손실은 추가 손실 항입니다. 물론, 더 많은 항들이 필요하지만, CycleGAN에서 주로 출력의 색상 보존을 돕기 위해 제안된 선택적 손실 항이며, 이것이 곧 어떻게 작동하는지 보게 될 것입니다.
<img src="./images/C3W3.45.png" width="700">
<br>

순환 일관성 손실 항의 적대적 손실 외에도 사용할 수 있는 또 다른 픽셀 차이 손실 항이 있습니다. 이것은 항등 손실이라고 불리는 선택 사항입니다. 항등 손실은 이미지의 색상을 보존하는 데 도움이 되며, 매핑에 있어서 본질적으로 더 합리적입니다. 이것이 의미하는 바는 이 말과 같은 이미지를 반대쪽 생성기, 즉 얼룩말에서 말로 가는 생성기에 넣음으로써 예측 결과를 보장한다는 것입니다. 이 실제 입력을 이 얼룩말에서 말로의 생성기에 입력하면 얼룩말이 아니라 이미 말 스타일이기 때문에 이상적으로 정확히 동일한 이미지를 출력해야 합니다. 그렇죠? 여기에서 얼룩말에서 말로 가는 생성기에서 항등 매핑이 적용되거나, 기본적으로 출력에 대한 입력의 변경이 없을 것으로 예상합니다. 손실 항에 대해 할 수 있는 일은 실제 입력과 생성기가 생성하는 모든 것, 출력 사이의 픽셀 거리를 얻고 이를 손실 함수에 추가할 수 있다는 것입니다. 픽셀 거리가 0인 이 경우 항등 손실은 0의 손실을 가지며 이것이 아이디어입니다. 이것이 바로 여러분이 생성기에 원하는 것입니다. 이미 말입니다. 생성기가 다른 것으로 변환하는 것을 원하지 않으므로 이 동작을 권장합니다.
<img src="./images/C3W3.46.png" width="700">
<br>

반대로 생성기가 이 말 이미지를 이상한 것으로 매핑하면 어떤 방식으로든 이미지가 변경됩니다. 예를 들어 생성기가 검정색과 흰색 색상을 갈색으로 매핑하기 때문에 색상이 변경될 수 있습니다. 그래서 아마도 다른 방법을 찾았을 것이고 일반적인 톤을 만들려고 할 것입니다. 입력 이미지를 다른 것으로. 여기에서 이 입력과 이 출력 사이의 픽셀 거리를 여기에서 취하고 이 매핑이 동일함 이외의 것이 되지 않도록 하고 싶습니다.
<img src="./images/C3W3.47.png" width="700">
<br>

다시 순환의 맥락에서, 이것은 생성자에 대한 전체 손실 함수입니다. 적대적 손실과 순환 일관성 손실에 더하여, 이제 말에 넣은 얼룩말과 말 생성자 모두에 대해 이 추가 항등 손실이 있습니다. 얼룩말에서 말 생성기 뿐아니라, 말에서 얼룩말 생성기에 대한 것도 추가합니다. 이것들이 함께 항등 손실이 되며,
<img src="./images/C3W3.48.png" width="700">
<br>

또는 이것을 구성하고 다시 $\lambda$ 항을 추가하려고 합니다. 저는 이것은 $\lambda_1$, 이것은 $\lambda_2$라고 하여, 순환 일관성 손실 항 및 항등 손실 항에 대해 다른 가중치라고 말할 것입니다.
<img src="./images/C3W3.49.png" width="700">
<br>

다음은 항등 손실이 CycleGAN을 안내하는 데 어떻게 도움이 되는지에 대한 예입니다. 입력으로 이러한 이미지가 있고 이 이미지가 다시, 다시 말해서 다른 유형의 풍경에 매핑하려고 한다고 가정해 보겠습니다. 항등 손실과 함께 이와 같은 항등 매핑을 권장하고 싶습니다. 항등 매핑을 기대한다고 가정해 보겠습니다. 그러나 항등 매핑이 없으면 이러한 유형의 색상 왜곡을 얻을 수 있습니다. 이러한 유형의 왜곡은 아마도 여름과 겨울 장면 사이를 매핑하려고 하고 겨울에 더 푸른 색조를 만들려고 하는 것으로 상상할 수 있습니다. 우리가 실제로 그 항등 매핑을 원할 때 이 동일항이 없으면 발생합니다.
<img src="./images/C3W3.50.png" width="700">
<br>

요약하면, 항등 손실(identity loss)은 그 더미 중 하나에서 실제 이미지를 가져와 반대쪽 생성기에 입력합니다. 생성기에 넣는 입력 이미지에는 생성기가 매핑하려는 스타일이 이미 있기 때문에 실제로 동일 매핑이 예상됩니다. 이를 결정하기 위해 픽셀 거리가 사용되며 이상적으로는 입력과 출력 간에 차이가 없습니다. 항등 손실은 0입니다. 항등 손실은 일부 작업에서 유색 보존에 도움이 되는 것으로 나타난 전체적 및 선택적 손실 항입니다. 이것이 선택 사항인 주된 이유는 많은 경우에 매우 도움이 되는 것으로 나타났지만, 다른 경우에는 별 차이가 없었기 때문입니다.
<img src="./images/C3W3.51.png" width="700">

### CycleGAN Putting It All Together
이제 CycleGAN을 모두 모아 합칠 시간입니다. 따라서 이것은 CycleGAN에 순환을 넣는 순환 일관성 손실이 있는 두 개의 서로 다른 GAN으로 구성됩니다. 주요 손실 항인 최소 제곱 적대적 손실과 선택적인 동일 손실 항입니다.
<img src="./images/C3W3.52.png" width="700">
<br>

먼저 여기에서 진짜 얼룩말에서 가짜 말까지 가는 과정을 보여주겠지만, 반대 방향으로도 마찬가지입니다. 시작하려면 먼저 얼룩말 이미지를 입력하고 얼룩말에서 말에 매핑되는 생성기를 사용하여 가짜 말을 얻습니다. 
<img src="./images/C3W3.53.png" width="700">
<br>

그런 다음 말 판별기는 이 가짜 이미지와 진짜 이미지를 모두 봅니다. 어느 것이 어느 것인지 알지 못하고 해당 이미지의 패치가 얼마나 진짜인지 또는 얼마나 가짜인지에 대한 분류 매트릭스를 출력합니다. 여기에 사용된 것은 최소 자승 손실입니다. 특히 진짜의 경우 이 분류 행렬은 1로 가득합니다. 그리고 가짜의 경우 0으로 가득 차 있고 이것이 최소 제곱 적대적 손실을 계산하는 방법입니다. 그리고 그것은 판별기를 위한 것이며, 생성기를 위해서는 이 분류 행렬은 실제로 모두 1입니다. 
<img src="./images/C3W3.54.png" width="700">
<br>

그리고, 이 최소 제곱 적대적 손실 외에도, 가짜 말을 가져와 말에서 얼룩말로 가는 다른 생성기를 통해 이 가짜 얼룩말을 생성하여 이 방향의 순환 일관성 손실을 계산할 수 있기를 원합니다. 그리고 실제 입력과 이 가짜 생성 얼룩말 사이의 픽셀 차이를 사용하여 이를 수행합니다. 두 생성기 간에 스타일을 전송해야 하기 때문에 실제로 동일하게 보여야 하기 때문입니다.
<img src="./images/C3W3.55.png" width="700">
<br>

그리고 다시, 반대 방향에서도 동일한 작업이 수행됩니다. 이것은 말과 얼룩말에서 나온 것입니다. 그런 다음 순환 일관성 손실에 대해 다시 말과 여기에 있는 얼룩말 판별기도 적용됩니다.
<img src="./images/C3W3.56.png" width="700">
<br>

작업에 동등 손실을 사용하기로 선택한 경우, 얼룩말을 말과 얼룩말의 반대 생성기에 넣고 싶을 수도 있습니다. 얼룩말을 얻으려면 여기에서 픽셀 차이를 가져와 항등 손실로 저장합니다. 
<img src="./images/C3W3.57.png" width="700">
<br>

얼룩말을 통해 말 생성기로 이동하는 말에 대해서도 동일한 작업을 수행해야 합니다.
<img src="./images/C3W3.58.png" width="700">
<br>

그리고 손실 함수에 항을 추가하는 것의 아름다움입니다. 너무 간단하면서도 매우 효과적입니다. CycleGAN에는 총 6개의 손실이 있습니다. 따라서 두 GAN 모두에 대한 최소 제곱 적대적 손실이 있습니다. 또한 순환 일관성 손실이 양방향으로 발생합니다. 그리고 마지막으로 거기에 있는 각 생성기에 대한 항등 손실이 있습니다. 그리고 그것은 전체 제너레이터 손실 함수를 구성하며 이는 두 제너레이터 모두에 해당합니다. 그래서 두 개의 생성기입니다. 한편, 각 판별자는 이들 중 하나에 초점을 맞추므로 이것은 말 판별자용이고 이것은 얼룩말 판별자용입니다.
<img src="./images/C3W3.59.png" width="700">
<br>

요약하자면 CycleGAN은 두 개의 GAN으로 구성된 다음 순환을 만들고 서로 의존하여 모든 유형의 손실 항을 계산하려고 합니다. 사실, 생성기에는 총 6개의 손실 항, 즉 최소 제곱 적대적 주요 손실 항과 순환 일관성 손실, 각 생성기에 대한 선택적 항등 손실. 그리고 판별자는 pix2pix에서 배운 PatchGAN을 사용하여 최소 제곱 적대적 손실로 조금 더 간단합니다.
<img src="./images/C3W3.60.png" width="700">


### CycleGAN 응용 및 변형
이제 CycleGAN에 대해 알게 되었으므로 이 섹션에서는 CycleGAN 응용 프로그램과 그 변형에 관한 것입니다. 따라서 먼저 CycleGAN의 일부 응용 프로그램을 확인한 다음, 짝을 이루지 않은 이미지 대 이미지 변환의 일부 변형을 볼 수 있습니다.
<img src="./images/C3W3.61.png" width="700">
<br>

따라서 CycleGAN은 Snapchat에서 본 다양한 필터, 노화, 인식된 성별 변경, 말에서 얼룩말로 변화, 장면의 다른 계절 변경 등 다양한 용도로 사용할 수 있습니다. 특정 그림 스타일을 만들어 이번 주 초에 본 사진에 이 모네처럼 스타일을 옮기거나 그 반대로 하면 됩니다. 이것은 또한 장면에서 개체를 매우 안정적으로 변경한다는 것을 의미합니다. 저는 이것에 대해 정말 매력적인 점이라고 생각합니다. 말의 gif-anim 으로부터 얼룩말로 변화에서 실제로 gif전체에 걸쳐 줄무늬를 매우 잘 유지하고 여기 울타리를 거의 방해하지 않는다는 것입니다.
<img src="./images/C3W3.62.png" width="700">
<br>

즉각적인 미디어 응용 프로그램 외에도 CycleGAN은 데이터 증강에도 자주 사용됩니다. 예를 들어, 지금은 종양이 있지만, 종양이 없었을 때의 사람의 쌍을 이루는 이미지를 얻는 것이 정말 어렵습니다. 그런 종류의 표현은 우리가 지금 당장 예측할 수 있는 것이 아닙니다. 그리고 이것은 여기에서 약간의 데이터 증강으로 얻을 수 있는 방법이 될 수 있습니다. 따라서 다운스트림 응용 프로그램을 위하여 종양을 표시한 것과 종양을 제거한 것의 쌍을 이루는 예를 얻을 수 있습니다. 
해당 종양의 성장 모니터링을 위한 종양 분할 및 분류는 의료 응용 분야에서 정말 중요합니다. 따라서 여기에서는 먼저 CycleGAN을 사용하여 종양인 이 큰 반점을 제거하고, 다음 여기에서는 종양을 추가하는 것을 볼 수 있습니다.
<img src="./images/C3W3.63.png" width="700">
<br>
그리고 다시 일반적인 데이터 증강의 경우 CycleGAN은 매우 현실적인 예를 생성할 수 있습니다. 예를 들어, 여기에 CT 스캔이 있고 올바른 분할을 수행한 전문가가 있습니다. Standard Augmentation을 사용하여 전혀 도달할 수 없는 반면, CycleGAN은 이러한 분할을 아주 잘 할수 있습니다. 사실, 여기 일반적 증강에서는 올바른 장소를 분할하고 있지도 않습니다.
<img src="./images/C3W3.64.png" width="700">
<br>

CycleGAN은 짝을 이루지 않은 이미지 대 이미지 변환을 수행할 수 있는 유일한 모델이 아닙니다. Unsupervised Image-to-Image Translation을 나타내는 UNIT라는 변형이 있습니다. 이 짝을 이루지 않은 이미지-이미지 변환은 레이블이 없기 때문에을 비지도로 생각할 수 있습니다. 그리고 이것은 자동차 운전의 밤 장면에서 낮 사이의 gif 입니다. 그리고 이 모델의 핵심 통찰력은 이 공유된 잠재 공간(shared latent space)로 알려져 있으며, 기본적으로 이 잠재 공간에 주어진 노이즈 벡터 $Z$가 있음을 나타냅니다. 예를 들어 도메인 $X_1$에서 이미지를 생성할 수 있으므로 해당 이미지를 생성할 수 있으며 해당 잠재성에 대한 매핑이 있습니다. 그리고 동일한 잠재성은 $X_2$라는 다른 도메인에 매핑할 수도 있고 해당 잠재성에 다시 매핑할 수도 있습니다. 그리고 이 두 도메인은 $X_1$이 주간 자동차 운전이고 이 $X_2$가 야간 도메인이라고 상상할 수 있습니다. 따라서 $X_1$과 $X_2$는 공유 잠재 공간의 스타일일 뿐입니다. 동일한 $Z$에서 동일한 링크를 공유하고 여전히 두 가지 다른 스타일을 생성할 수 있습니다. 우리는 같은 내용을 풀고 한 방향이 아니라 앞뒤로 매핑할 수 있는 두 가지 다른 스타일을 계속 생성할 수 있습니다.
<img src="./images/C3W3.65.png" width="700">
<br>

그리고 UNIT를 한 단계 더 나아가서 Multimodal UNIT 또는 MUNIT이라는 것이 있으며, multimodal이 의미하는 바는 스케치의 한 영역에서 두 번째 영역의 많은 모드로 이동할 수 있다는 것입니다. 따라서 MUIT는 이 하나뿐 아니라 다른 모든 항목을 매핑하는 것도 찾을 수 있습니다. 그리고 정말 멋진 점은 예를 들어 신발 더미에 있는 이러한 모든 다양한 스타일에 대해 모델에게 실제로 말하지 않는다는 것입니다. 그것은 당신이 이 모든 다른 신발 스타일을 가지고 있다는 것을 알아낼 수 있고 이것을 하나뿐만 아니라 많은 것에 매핑할 수 있습니다. 그리고 다시, 이것은 감독되지 않기 때문에 정말 멋집니다. 신발의 차이점이 무엇인지 라벨을 붙일 필요는 없습니다. 그리고 간단히 설명하자면 이러한 기술 UNIT 및 MUNIT 모두 이미지 또는 스타일 인코딩에 의존합니다. 그래서 그들은 사실적인 생성을 위해 GAN 구성 요소를 사용하면서 VAE 측면에서 영감을 얻고 있습니다.
<img src="./images/C3W3.66.png" width="700">
<br>

요약하면, CycleGAN의 확장은 UNIT 및 MUNIT를 포함하여 존재하며, 이는 공유 잠재 공간 가정과 MUNIT의 경우 다중 모드 생성을 사용합니다. 이는 꽤 멋진 일입니다. 특히 예술, 의료 분야, 비디오 및 비디오 게임에서 CycleGAN의 많은 응용 프로그램이 있습니다. 
<img src="./images/C3W3.67.png" width="700">